# Load libraries

In [49]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
import plotly.express as px

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, homogeneity_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_blobs

from scipy.cluster.hierarchy import dendrogram, linkage


import warnings
import os
warnings.filterwarnings("ignore")

py.offline.init_notebook_mode(connected = True)

# Data Exploration

In [50]:
df = pd.read_csv('transactions.csv')
df.head()

,customer_id,trans_date,trans_amount
0,CS5295,11-Feb-18,35
1,CS4768,15-Mar-20,39
2,CS2122,26-Feb-18,52
3,CS1217,16-Nov-16,99
4,CS1850,20-Nov-18,78


In [51]:
df.shape

(125000, 3)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125000 entries, 0 to 124999
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   customer_id   125000 non-null  object
 1   trans_date    125000 non-null  object
 2   trans_amount  125000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.9+ MB


In [53]:
df.head()

,customer_id,trans_date,trans_amount
0,CS5295,11-Feb-18,35
1,CS4768,15-Mar-20,39
2,CS2122,26-Feb-18,52
3,CS1217,16-Nov-16,99
4,CS1850,20-Nov-18,78


In [54]:
df.describe()

,trans_amount
count,125000.000000
mean,64.991912
std,22.860006
min,10.000000
25%,47.000000
50%,65.000000
75%,83.000000
max,105.000000


# Data Analysis

In [55]:
#To change the date from object to datetime

df["trans_date"] = pd.to_datetime(df["trans_date"], dayfirst=True, errors='coerce')
df.head()

,customer_id,trans_date,trans_amount
0,CS5295,2018-02-11,35
1,CS4768,2020-03-15,39
2,CS2122,2018-02-26,52
3,CS1217,2016-11-16,99
4,CS1850,2018-11-20,78


In [56]:
df.isnull().any()

customer_id     False
trans_date       True
trans_amount    False
dtype: bool

In [57]:
df['trans_date'] = df['trans_date'].replace(np.nan, '2017-02-28')

In [58]:
df['trans_date'] = df['trans_date'].astype('datetime64[ns]')

# RFM Analysis

- Recency: how recent was the customer's last purchase
- Frequency: how often did the customer make a purchase in a given period
- Monetary: how much money did the customer spend in a given period

## 1. Recency

In [59]:
import datetime as dt

In [60]:
now = dt.datetime(2021,8,28)

In [61]:
recency = df.groupby(by='customer_id', as_index=False)['trans_date'].max()
recency['recency'] = now - recency['trans_date']

In [62]:
recency.head()

,customer_id,trans_date,recency
0,CS1112,2020-01-14,592 days
1,CS1113,2020-02-09,566 days
2,CS1114,2020-02-12,563 days
3,CS1115,2020-03-05,541 days
4,CS1116,2019-08-25,734 days


## 2. Frequency

In [63]:
frequency = df.groupby(by=['customer_id'], as_index=False)['trans_date'].count()

In [64]:
frequency.rename(columns={'trans_date':'frequency'}, inplace=True)

In [65]:
frequency.head()

,customer_id,frequency
0,CS1112,15
1,CS1113,20
2,CS1114,19
3,CS1115,22
4,CS1116,13


## 3. Monetery

In [66]:
monetery = df.groupby(by=['customer_id'], as_index=False)['trans_amount'].sum()

In [67]:
monetery.rename(columns={'trans_amount':'monetery'}, inplace=True)

In [68]:
monetery.head()

,customer_id,monetery
0,CS1112,1012
1,CS1113,1490
2,CS1114,1432
3,CS1115,1659
4,CS1116,857
